In [ ]:
import torch, torchvision

In [ ]:
transforms = torchvision.transforms.Compose([
                torchvision.transforms.ToTensor()
])
training_dataset = torchvision.datasets.MNIST(root = 'mnistdata', train = True,download=True, transform = transforms)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnistdata/MNIST/raw/train-images-idx3-ubyte.gz to mnistdata/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnistdata/MNIST/raw/train-labels-idx1-ubyte.gz to mnistdata/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnistdata/MNIST/raw/t10k-images-idx3-ubyte.gz to mnistdata/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnistdata/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnistdata/MNIST/raw



In [ ]:
test_dataset = torchvision.datasets.MNIST(root = 'mnistdata', train=False, transform = transforms, download=True)

In [ ]:
test_x_feature_vectors, train_x_feature_vectors = [], []
for test_index in range(len(test_dataset)):
  test_x_feature_vectors.append((torch.flatten(test_dataset[test_index][0])))
for train_index in range(len(training_dataset)):
  train_x_feature_vectors.append((torch.flatten(training_dataset[train_index][0])))

In [ ]:
test_x_feature_vectors[0].shape, train_x_feature_vectors[0].shape

(torch.Size([784]), torch.Size([784]))

In [ ]:
torch_test_fvecs = torch.stack(test_x_feature_vectors)

In [ ]:
torch_test_fvecs.shape

torch.Size([10000, 784])

In [ ]:
torch_train_fvecs = torch.stack(train_x_feature_vectors)

In [ ]:
torch_train_fvecs.shape

torch.Size([60000, 784])

In [ ]:
import numpy as np 
listofdistances = [] #10000 x 60000 should be end outcome
def kNN_distance_matrix(test_fvecs, train_fvecs, distance_list):
  for test_index in range(test_fvecs.shape[0]):
    distance_vector = torch.square(test_fvecs[test_index] - train_fvecs).sum(dim=1)
    listofdistances.append(distance_vector)
  return np.array(listofdistances) 
distance_matrix = kNN_distance_matrix(torch_test_fvecs, torch_train_fvecs, listofdistances)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [ ]:
distance_matrix.shape

(10000,)

In [ ]:
len(distance_matrix)

10000

In [ ]:
def Extract_closest_k_indices(knn_distance_matrix, k):
  list_of_k_closest = [] # Should end up being 10000 x k 
  for test_indice in range(len(knn_distance_matrix)):
    list_of_k_closest.append(np.argpartition(knn_distance_matrix[test_indice], k)[:k])
  return np.array(list_of_k_closest)

In [ ]:
k_closest_indices = Extract_closest_k_indices(distance_matrix, 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


In [ ]:
torch.save(k_closest_indices, '/content/drive/MyDrive/A1 2/k_closest_indices.pth')

Now we are reloading the indices that we saved previously in our google drive as well as re running the code that gave us our mnist dataset

In [ ]:
k_nearest_indices = torch.load('/content/drive/MyDrive/A1 2/k_closest_indices.pth')

In [ ]:
k_nearest_indices[0]

tensor([16186, 38620, 53843, 27059, 47003])

Now the idea is that we will use the k nearest indices of each test example and find the instances in the training set that have these indices and then take a max voting from all of these values from the training set

In [ ]:
k_nearest_indices.shape

(10000,)

In [ ]:
#returns accuracy
def find_k_nearest_classes(k_nearest_indices, train):
  list_of_k_nearest_classes = []
  for row in range(len(k_nearest_indices)):
    curr_list = []
    for column in range(len(k_nearest_indices[row])):
      curr_list.append(train[k_nearest_indices[row][column]][1])
    list_of_k_nearest_classes.append(curr_list)
  return list_of_k_nearest_classes

In [ ]:
k_list = find_k_nearest_classes(k_nearest_indices, training_dataset)

In [ ]:
k_list[2]

[1, 1, 1, 1, 1]

In [ ]:
import scipy.stats 
final_classes = []
for row in range(len(k_list)):
  final_classes.append(scipy.stats.mode(k_list[row])[0][0])

In [ ]:
len(final_classes)

10000

In [ ]:
def accuracy_of_kNN_Classifier(final_classes, test):
  count = 0
  for i in range(len(final_classes)):
    if final_classes[i] == test[i][1]:
      count +=1
  return count/len(final_classes)

In [ ]:
accuracyofkNNClassifier = accuracy_of_kNN_Classifier(final_classes, test_dataset)

In [ ]:
print(accuracyofkNNClassifier) #We got 96 percent accuracy with kNN Classifier on MNIST!

0.9688
